In [1]:
from PIL import Image
import numpy as np
import os

layerList = ["가", "나", "다", "라", "마", "바", "아", "자", "차", "카", "타", "파", "하"]

all_x_array = []
all_y_array = []

for dir in layerList:
    fileList = os.listdir(f"../dataset_dir/new_image/{dir}")

    for file in fileList:
        y_array = dict.fromkeys(layerList, 0) # 12000개의 str 값을 0으로 초기화하는 dict 생성

        imageData = Image.open(f"""../dataset_dir/new_image/{dir}/{file}""") # 이미지 tensor 데이터 출력
        imageData = imageData.resize((50, 50)) # 이미지 업스케일링

        imageData = np.array(imageData)
        all_x_array.append(imageData.tolist())

        y_array[dir] = 1 # 사용자가 지정한 str 값을 1로 변경
        y_array = list(y_array.values())
        all_y_array.append(y_array)


In [2]:
x_array = np.array(all_x_array)
y_array = np.array(all_y_array)

In [3]:
print(x_array.shape, y_array.shape)

(80988, 50, 50) (80988, 13)


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    x_array, y_array, test_size=0.2, shuffle=False, random_state=1004
)

In [5]:
X_train = X_train.astype("float32") / 255
Y_train = Y_train.astype("float32") / 255

X_test = X_test.astype("float32") / 255
Y_test = Y_test.astype("float32") / 255

In [6]:
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(64790, 50, 50) (16198, 50, 50)
(64790, 13) (16198, 13)


In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [9]:
import tensorflow as tf
import keras

model = keras.Sequential([
  keras.layers.Input((50, 50, 1)),
  keras.layers.Conv2D(32, (3, 3), activation='relu'),
  keras.layers.Conv2D(32, (3, 3), activation='relu'),
  keras.layers.MaxPooling2D((2, 2), (2, 2)),

  # 2단계 블록
  keras.layers.Conv2D(64, (3, 3), activation='relu'),
  keras.layers.Conv2D(64, (3, 3), activation='relu'),
  keras.layers.MaxPooling2D((2, 2), (2, 2)),

  # 3단계 블록
  keras.layers.Conv2D(128, (3, 3), activation='relu'),
  keras.layers.Conv2D(128, (3, 3), activation='relu'),
  keras.layers.Conv2D(128, (3, 3), activation='relu'),
  keras.layers.MaxPooling2D((2, 2), (2, 2)),

  # 4단계 블록
  keras.layers.Conv2D(128, (3, 3), activation='relu'),
  keras.layers.Conv2D(128, (3, 3), activation='relu'),
  keras.layers.Conv2D(128, (3, 3), activation='relu'),
  keras.layers.MaxPooling2D((2, 2), (2, 2)),

  # 전결합층
  keras.layers.Flatten(),
  keras.layers.Dense(32, activation='relu'),
  keras.layers.Dropout(0.2),
  keras.layers.Dense(16, activation='relu'),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(13, activation='softmax'),
])

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_20. Consider increasing the input size. Received input shape [None, 1, 1, 128] which would produce output shape with a zero or negative value in a dimension.

In [ ]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [ ]:
batch_size = 128
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(0.01), metrics=["accuracy"])

history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test))

Epoch 1/20


ValueError: in user code:

    File "/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.8/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filek0wod35f.py", line 10, in tf__call
        x = ag__.converted_call(ag__.ld(self).one_stap_conv, (ag__.ld(inputs),), None, fscope)

 